In [49]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [50]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_a280 = pd.read_csv("tsp_instances_txt/indice_a280.csv");
df_bayg29 = pd.read_csv("tsp_instances_txt/indice_bayg29.csv");# el csv lo arme a mano, sin celdas
df_berlin52 = pd.read_csv("tsp_instances_txt/indice_berlin52.csv");
#df_densidad_baja = pd.read_csv("instancias/densidad-baja/indice.csv");

In [51]:
df_a280

,dataset,n,m,archivo
0,a280,280,39060,tsp_instances_txt/a280.txt


In [52]:
df_bayg29

,dataset,n,m,archivo
0,bayg29,29,406,tsp_instances_txt/bayg29.txt


In [53]:
df_berlin52

,dataset,n,m,archivo
0,berlin52,52,1326,tsp_instances_txt/berlin52.txt


In [58]:
def correr_experimento(metodo, mem_size, porc_vecindad, iteraciones, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../../tp2_tspAlgo3/tp2", metodo, mem_size, porc_vecindad, iteraciones], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    costo_ciclo = int(process.stdout.read());
    print(costo_ciclo)
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    instancia = (tiempo_de_ejecucion, costo_ciclo)
    return instancia;

In [55]:
experimentos = [];

# Experimento 1
Experimentos para el metodo TSE-AGM, variando el tamaño de la memoria

In [56]:
#for n in range(0, 10):
#    fila_n = df_a280.iloc[n];
#    experimentos.append([fila_n["dataset"], n, fila_n["m"], "TSE-AGM", fila_n["archivo"]]);

## varia tamaño de vecindad:
fila_n = df_a280.iloc[0];
memSize = 20

#porcentajeVecindad
iteraciones = 1000
rango_porcentajes = np.array(np.arange(10, 60, 10)) 
for p in rango_porcentajes: # 15 elementos
    experimentos.append([fila_n["dataset"], "variando-vecinos", fila_n["n"], fila_n["m"], "TSE-AGM", memSize, p, iteraciones, fila_n["archivo"]]);
    
## varia tamaño de memoria:
porcentaje_vecindad = 10
rango_memoria = np.array(np.arange(10, 60, 10))
for t in rango_memoria : # 15 elementos
    experimentos.append([fila_n["dataset"], "variando-memoria", fila_n["n"], fila_n["m"], "TSE-AGM", t, porcentaje_vecindad, iteraciones, fila_n["archivo"]]);

## varia cantidad de iteraciones:
rango_iteraciones = np.array(np.arange(10, 1000, 10)) 
for i in rango_iteraciones: # 20 elementos
    experimentos.append([fila_n["dataset"], "variando-iteraciones", fila_n["n"], fila_n["m"], "TSE-AGM", memSize, porcentaje_vecindad, i, fila_n["archivo"]]);



## Cargo solo una instancia, el resto son repeticiones

In [27]:
#fila_n = df_a280.iloc[0];
fila_n = df_bayg29.iloc[0]; # lee una instancia
experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["m"], "TSE-AGM", fila_n["archivo"]]); # decia n en lugar de fila_n["n"]

In [41]:
print(fila_n)

dataset                          a280
n                                 280
m                               39060
archivo    tsp_instances_txt/a280.txt
Name: 0, dtype: object


In [29]:
rango = np.array(np.arange(10, 51, 10))
print(rango)

[10 20 30 40 50]


# Experimento 2: BERLIN52
Experimentos para el metodo TSE-AGM, variando el tamaño de la vecindad, memoria, iteraciones (por separado)

In [34]:
#contenido de df_berlin52: dataset,n,m,archivo

## varia tamaño de vecindad:
fila_n = df_berlin52.iloc[0];
memSize = 20
#porcentajeVecindad
iteraciones = 1000
rango = np.array(np.arange(10, 51, 10))
for p in [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60]: # 15 elementos
    experimentos.append([fila_n["dataset"], "variando-vecinos", fila_n["n"], fila_n["m"], "TSE-AGM", memSize, p, iteraciones, fila_n["archivo"]]);
    
## varia tamaño de memoria:
porcentaje_vecindad = 10
for t in [1,5,10,20,30,40,50,100,150,200,300,400,500,600,700]: # 15 elementos
    experimentos.append([fila_n["dataset"], "variando-memoria", fila_n["n"], fila_n["m"], "TSE-AGM", t, porcentaje_vecindad, iteraciones, fila_n["archivo"]]);

## varia cantidad de iteraciones:
for i in [1,5,10,20,30,40,50,70,80,90,100,200,300,400,500,600,700,800,900,1000]: # 20 elementos
    experimentos.append([fila_n["dataset"], "variando-iteraciones", fila_n["n"], fila_n["m"], "TSE-AGM", memSize, porcentaje_vecindad, i, fila_n["archivo"]]);



In [35]:
#columnas de experimento[i]: dataset, caso, n, m, metodo, memSize, porcentajeVecindad, iteraciones, archivo

In [43]:
len(experimentos)

210

## (EMPROLIJAR)
## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
lo pruebo con berlin52

In [57]:
columnas = ["dataset", "caso", "n", "m", "metodo", "memSize","porcentajeVecindad","iteraciones","tiempo","costo"];
filas = [];
numero = 1
#porcentaje_vecindad = 30 #CAMBIO
#iteraciones = 30 #CAMBIO
T = 1 #5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    costo = 0
    #for i in range(0, T):
    #    res = correr_experimento(experimento[3],str(t),str(porcentaje_vecindad),experimento[4]);#CAMBIO 
    #    tiempos.append(res[0]);
    #    costo = res[1]
    
    #parametros: correr_experimento(metodo, mem_size, porc_vecindad, iteraciones, archivo_instancia)
    #columnas de experimento[i]: dataset, caso, n, m, metodo, memSize, porcentajeVecindad, iteraciones, archivo
    res = correr_experimento(experimento[4],str(experimento[5]),str(experimento[6]), str(experimento[7]),experimento[8]);#CAMBIO 
    tiempos.append(res[0]);
    costo = res[1]
    
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], experimento[4], experimento[5], experimento[6], experimento[7], tiempo, costo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 109/109'

entre
2691
